# 08/03/2025

Задание 1: Эхо-сервер (TCP)
Цель: Научиться создавать простое клиент-серверное взаимодействие через TCP.
Задача:
1. Реализуйте TCP-сервер, который:
o Принимает подключение клиента.
o Получает от клиента строку.
o Отправляет обратно ту же строку (эффект "эхо").
o Завершает соединение после ответа.
2. Реализуйте TCP-клиента, который:
o Подключается к серверу.
o Отправляет любое сообщение.
o Получает и выводит ответ сервера.
3. Дополнительно: Измерьте время работы клиента — от подключения до получения
ответа.
Пример работы:
Клиент отправил: Привет, сервер!
Сервер ответил: Привет, сервер!

In [6]:
import socket
import threading
import time

# Задание адреса и порта для прослушивания
HOST = ''  # Все интерфейсы
PORT = 65432  # Порт для прослушивания

def main():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        try:
            # Привязка сокета к адресу и порту
            server_socket.bind((HOST, PORT))
            server_socket.listen()
            print(f"Слушаю на {HOST}:{PORT}")
        
            timer_thread = threading.Timer(6, stop_server)  # Таймер на 6 секунд
            timer_thread.start()  # Запуск таймера

            while True:
                conn, addr = server_socket.accept()  # Принятие нового соединения
                handle_client(conn, addr)
        except KeyboardInterrupt:
            print("\nСервер остановлен.")

def handle_client(conn, addr):
    print(f"Принято новое соединение от {addr}")
    with conn:
        while True:
            data = conn.recv(1024).decode('utf-8')  # Получение данных от клиента
            if not data:
                break
            print(f"Принял данные: {data}")
            conn.sendall(data.encode('utf-8'))  # Ответ клиенту (echo)
            print(f"Отправил данные обратно: {data}")

def stop_server():
    """Функция остановки сервера"""
    print("Таймер истек, сервер останавливается.")
    exit(0)  # Завершение выполнения программы

if __name__ == "__main__":
    main()

Слушаю на :65432
Таймер истек, сервер останавливается.


: 

In [2]:
import socket
import time

# Задание адреса и порта сервера
SERVER_HOST = 'localhost'  # Или IP-адрес сервера
SERVER_PORT = 65432  # Тот же порт, что используется сервером

def main():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        try:
            client_socket.connect((SERVER_HOST, SERVER_PORT))
            message = input("Введите сообщение для отправки серверу: ")
            client_socket.sendall(message.encode('utf-8'))

            response = client_socket.recv(1024).decode('utf-8')
            print(f"Сервер ответил: {response}")
        except ConnectionRefusedError:
            print(f"Не удалось подключиться к серверу {SERVER_HOST}:{SERVER_PORT}. Проверьте, запущен ли сервер.")

if __name__ == "__main__":
    main()

Не удалось подключиться к серверу localhost:65432. Проверьте, запущен ли сервер.


In [2]:
import time

def main():
    while True:
        print("Программа работает...")
        time.sleep(3)  # Пауза на 3 секунд
        break  # Прерывание цикла и завершение процесса
    print("Процесс остановлен.")

if __name__ == "__main__":
    main()

Программа работает...
Процесс остановлен.


Задание 2: Многопоточный чат (TCP)
Цель: Освоить многопоточное программирование с использованием сокетов для
поддержки нескольких клиентов одновременно.
Задача:
1. Напишите TCP-сервер, который:
o Поддерживает несколько подключений одновременно с помощью потоков.
o Принимает сообщения от каждого клиента.
o Отправляет полученное сообщение всем остальным подключенным
клиентам (эффект группового чата).
2. Напишите TCP-клиента, который:
o Подключается к серверу.
o Отправляет и получает сообщения, пока пользователь не введет "exit".
3. Дополнительно: Реализуйте возможность приватного чата (команды вида @user:
Привет!).
Пример работы:
[User1]: Привет, это чат!
[User2]: Привет! Как дела?

In [5]:
def handle_client(conn, addr):
    # Получаем имя пользователя
    conn.sendall('Пожалуйста, введите ваше имя пользователя: '.encode('utf-8'))
    username = conn.recv(1024).decode('utf-8').strip()
    
    # Регистрируем пользователя
    with lock:
        clients[username] = conn
        print(f'[NEW USER] {username} joined.')
        broadcast(f'{username} присоединился к чату.', exclude=[conn])

    try:
        while True:
            data = conn.recv(1024).decode('utf-8')
            if not data:
                break
            
            if '@' in data and ':' in data:
                # Это приватное сообщение
                target_username, message = data.split(':')[0].strip(), ':'.join(data.split(':')[1:])
                target_user = target_username.strip('@')
                if target_user in clients:
                    clients[target_user].sendall(f'[PRIVATE MESSAGE FROM {username}] {message}'.encode('utf-8'))
                else:
                    conn.sendall(f'Пользователь {target_user} не найден.'.encode('utf-8'))
            else:
                # Это публичное сообщение
                broadcast(f'[{username}] {data}', exclude=[conn])
    except ConnectionResetError:
        pass
    finally:
        with lock:
            if username in clients:
                del clients[username]
                broadcast(f'{username} покинул чат.', exclude=[conn])
        conn.close()
        print(f'[DISCONNECTED] {addr}')

In [2]:
def broadcast(message, exclude=None):
    if exclude is None:
        exclude = []
    for user_conn in clients.values():
        if user_conn not in exclude:
            user_conn.sendall(message.encode())

In [3]:
def send_message():
    while True:
        message = input('')
        if message.startswith('/'):
            # Обработка команд
            command = message.split()[0][1:]
            if command == 'help':
                print_help()
            elif command == 'list_users':
                list_users()
            elif command == 'private':
                private_message(message)
            else:
                print('Неизвестная команда. Используйте /help для получения списка команд.')
        elif message.lower().strip() == 'exit':
            break
        else:
            client_socket.sendall(message.encode())
    client_socket.close()
    sys.exit()

In [4]:
def connect_to_server():
    global client_socket
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((HOST, PORT))
        print('Подключение к серверу установлено.')
    except Exception as e:
        print(f'Ошибка подключения к серверу: {str(e)}')
        return

In [5]:
def receive_messages():
    while True:
        try:
            client_socket.settimeout(1)  # Установка таймаута в 1 секунду
            data = client_socket.recv(BUFFER_SIZE).decode('utf-8')
            if not data:
                continue
            print(data)
        except socket.timeout:
            continue
        except OSError:
            break

Задание 3: Сервер времени (UDP)
Цель: Научиться работать с UDP и понять разницу с TCP.
Задача:
1. Реализуйте UDP-сервер, который:
o Слушает порт.
o Отвечает текущим временем в формате: HH:MM:SS.
2. Реализуйте UDP-клиента, который:
o Отправляет запрос серверу с текстом "time".
o Получает и выводит текущее время от сервера.
3. Дополнительно: Измерьте задержку между отправкой запроса и получением
ответа.
Пример работы:
Запрос клиента: time
Ответ сервера: 14:05:32

In [1]:
import socket
import datetime

# Настройки сервера
SERVER_HOST = "127.0.0.1"
SERVER_PORT = 12345


def udp_time_server():
    try:
        # Создаем UDP-сокет
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as server_socket:
            server_socket.bind((SERVER_HOST, SERVER_PORT))

            print(f"Сервер времени прослушивает порт {SERVER_PORT}.")

            while True:
                # Ожидаем запрос от клиента
                request_data, address = server_socket.recvfrom(1024)

                # Преобразуем данные в строку
                request_text = request_data.decode("utf-8").strip()

                if request_text == "time":
                    # Получаем текущее время
                    current_time = datetime.datetime.now().strftime("%H:%M:%S")
                    response_data = f"Текущее время: {current_time}"

                    # Отправляем ответ клиенту
                    server_socket.sendto(response_data.encode("utf-8"), address)

                    print(f"Запрошено время от {address}, ответ: {response_data}")
                else:
                    print(f"Поступил неизвестный запрос '{request_text}' от {address}")
    except KeyboardInterrupt:
        print("\nПрервано пользователем.")
    except Exception as e:
        print(f"Ошибка: {str(e)}")


if __name__ == "__main__":
    udp_time_server()

Сервер времени прослушивает порт 12345.


: 

In [1]:
import socket
import time

# Настройки клиента
CLIENT_HOST = "127.0.0.1"
CLIENT_PORT = 12345


def udp_time_client():
    try:
        # Создаем UDP-сокет
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as client_socket:
            # Отправляем запрос серверу
            request_data = b"time"
            client_socket.sendto(request_data, (CLIENT_HOST, CLIENT_PORT))

            # Ждем немного, чтобы сервер успел обработать запрос
            time.sleep(0.1)

            # Измеряем время отправки запроса
            start_time = time.time()

            # Получаем ответ от сервера
            response_data, _ = client_socket.recvfrom(2048)  # Увеличили размер буфера

            # Измеряем время получения ответа
            end_time = time.time()

            # Расчет задержки
            delay = end_time - start_time

            # Преобразуем данные в строку
            response_text = response_data.decode("utf-8").strip()

            print(f"Полученный ответ: {response_text}")
            print(f"Задержка: {delay:.6f} секунд")
    except KeyboardInterrupt:
        print("\nПрервано пользователем.")
    except Exception as e:
        print(f"Ошибка: {str(e)}")


if __name__ == "__main__":
    udp_time_client()

Ошибка: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
